In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
import itertools,pandas,io

def Possible_match_pair(seller,buyer):    
    match_dict = []
    y = list(itertools.product(seller, repeat=len(buyer)))
    
    for i in y:
        this_dic = dict()
        for j in range(len(i)): 
            this_dic[buyer[j]] = i[j]  
        match_dict.append(this_dic)
    return match_dict

def reduce_match_pair_by_quantity(buyer_csv_info,seller_inventory,match_dict):
    valid_pair = []
    for keys in match_dict:        
        inventory = seller_inventory.copy()
        temp_list = []
        
        for key,value in keys.items():
            if buyer_csv_info[key]<=inventory[value]:
                inventory[value] -= buyer_csv_info[key]
                temp_list.append(True)
            else:
                temp_list.append(False)
              
        if all(temp_list):
            valid_pair.append(keys)
    return valid_pair

def get_cost(valid_pair,buyer_csv_info,seller_price,seller_discount,seller_minamt):
    price = []  
    
    for i in valid_pair:
        flipped = {}
        total_price = 0
        discount = 0
        for key, value in i.items():
            if value not in flipped:
                flipped[value] = [buyer_csv_info[key]*seller_price[value]]
            else:
                 flipped[value] += [buyer_csv_info[key]*seller_price[value]]
                
        print(flipped)        
                
        for k,v in flipped.items():
            if seller_minamt[k]<=sum(v):
                discount += int(seller_discount[k].strip("%\n "))
                total_price += sum(v)-(sum(v)*(int(seller_discount[k].strip("%\n "))/100)) 
            else:
                total_price+=sum(v)
        price.append({"match pair":str(i),"price":total_price,"discount":discount})
    return price    

def bestpair(price):
    #search the best individual among population.
    best_pair = price[0]
    for i in range(len(price)):
        if best_pair["price"]>=price[i]["price"]:
            best_pair = price[i]        
    return best_pair  
  
def main():
    match_dict = []
    seller_data = []
    buyer_data =[]
    valid_pair =[]
    #reading from the  buyer csv 

    buyer = pandas.read_csv(io.StringIO(uploaded['buyers.csv'].decode('utf-8')))
    buyer_csv_info = dict(zip(buyer.id.tolist(),buyer.quantity.tolist()))
    buyer_data = buyer.id.tolist()

    #reading from seller csv    
    seller_csv = pandas.read_csv(io.StringIO(uploaded['sellers.csv'].decode('utf-8')))
    seller_csv_info = seller_csv.to_dict()
    seller_inventory = dict(zip(seller_csv.id.tolist(),seller_csv.inventory.tolist()))
    seller_discount = dict(zip(seller_csv.id.tolist(),seller_csv.discount.tolist()))
    seller_price = dict(zip(seller_csv.id.tolist(),seller_csv.price.tolist()))
    seller_minamt = dict(zip(seller_csv.id.tolist(),seller_csv.minAmount.tolist()))
    seller_data = seller_csv.id.tolist()
    
    #check if the buyer total order quantity is less than or equal to the seller total inventory quantity.
    if buyer.quantity.sum() <= seller_csv.inventory.sum():
        #generate all possible outcome using cartesian product
        match_dict=Possible_match_pair(seller_data,buyer_data)
        print(len(match_dict))
#         for i in match_dict:
#             print(i)
        #search all the valid pair 
        valid_pair = reduce_match_pair_by_quantity(buyer_csv_info,seller_inventory,match_dict)
        print(len(valid_pair))
#         for i in valid_pair:
#             print(i)
        #calculate the price and  discount
        if len(valid_pair)>0:
          price = get_cost(valid_pair,buyer_csv_info,seller_price,seller_discount,seller_minamt)
          for i in price:
            print(i)
          best_pair = bestpair(price) 
          print("best pair",best_pair)
        else:
          print("there is no possible valid pair")
       
#         df = pandas.DataFrame(price)
#         df.to_csv('list.csv', index=False)

if __name__ == "__main__":
    main()
